In [1]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 단파장 비율 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('./../swr_ratio_classification_test.csv', delimiter=',', dtype='float')

x_data = []
for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][0])
    x_data.append(temp)
    
# scaler = MinMaxScaler(feature_range=(0, 1))
# x_data = scaler.fit_transform(x_data)
test_x = x_data[86:]
x_data = x_data[:86]

y_data = []
test_y = []

for i in range(86):
    temp = []
    temp.append(import_data[i][1])
    y_data.append(temp)

raw_y = y_data
    
for i in range(86, len(import_data)):
    temp = []
    temp.append(import_data[i][1])
    test_y.append(temp)
    
# raw_y = y_data

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder("float", [None, 1])
Y = tf.placeholder("float", [None, 5])

nb_classes = 5

# W = tf.Variable(tf.random_normal([1, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[1, nb_classes], initializer=xavier_init(1, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

with tf.Session() as sess:
    accuracy = 0
    sess.run(tf.global_variables_initializer())

    for step in range(200001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 5000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))

    print('-----------------------------')

    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

    for i in range(len(test_x)):
        a = sess.run(H, feed_dict={X:[test_x[i]]})
        print(sess.run(tf.argmax(a, 1)), test_y[i])
        if(sess.run(tf.argmax(a, 1)) ==test_y[i]):
            accuracy += 1
    print("accuracy = ", float(accuracy / len(test_y) * 100))

[[20.42], [21.28], [20.0], [20.83], [20.17], [20.31], [21.89], [20.67], [20.54], [20.71], [23.66], [21.45], [21.08], [21.33], [21.51], [21.53], [22.24], [21.77], [20.51], [21.01], [22.18], [22.07], [21.74], [23.21], [21.56], [22.48], [21.66], [23.98], [23.15], [23.1], [20.86], [23.76], [20.95], [20.84], [21.88], [22.1]]
[[2.0], [2.0], [2.0], [2.0], [2.0], [2.0], [2.0], [2.0], [2.0], [2.0], [4.0], [2.0], [2.0], [2.0], [2.0], [2.0], [3.0], [2.0], [2.0], [2.0], [3.0], [3.0], [2.0], [4.0], [2.0], [3.0], [2.0], [4.0], [4.0], [4.0], [2.0], [4.0], [2.0], [2.0], [2.0], [3.0]]
0 20.799513
5000 1.2873151


KeyboardInterrupt: 

In [ ]:
70.93%
86.11%